In [1]:
! pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [3]:
import pandas as pd

paper_names = [f"P{str(i).zfill(3)}.pdf" for i in range(1, 16)]
labels = ["Non-Publishable"] * 5 + ["Publishable"] * 10

df = pd.DataFrame({"Paper_Name": paper_names, "Label": labels})
df.to_csv("labeled_papers.csv", index=False)

print("✅ labeled_papers.csv created!")


✅ labeled_papers.csv created!


In [4]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()

# Create the 'papers' folder
os.makedirs("papers", exist_ok=True)

# Unzip the uploaded file into papers/
for fname in uploaded.keys():
    with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall("papers")

print("✅ All PDFs extracted to /papers")


Saving Papers-20250417T102120Z-001.zip to Papers-20250417T102120Z-001.zip
✅ All PDFs extracted to /papers


In [5]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()

# Create the 'papers' folder
os.makedirs("refrence", exist_ok=True)

# Unzip the uploaded file into papers/
for fname in uploaded.keys():
    with zipfile.ZipFile(fname, 'r') as zip_ref:
        zip_ref.extractall("refernce")

print("✅ All PDFs extracted to /reference_papers")

Saving Reference-20250417T102131Z-001.zip to Reference-20250417T102131Z-001.zip
✅ All PDFs extracted to /reference_papers


In [19]:
import os
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from sentence_transformers import util
from PyPDF2 import PdfReader
from tqdm import tqdm

paper_names = []
for i in range(1, 6):
    labeled_pdf_dir = "refernce/Reference/Non-Publishable"
    paper_names.append(f"{labeled_pdf_dir}/R{str(i).zfill(3)}.pdf")

# Publishable papers: R006 to R015
for i in range(6, 16):
    labeled_pdf_dir = "refernce/Reference/Publishable"
    paper_names.append(f"{labeled_pdf_dir}/R{str(i).zfill(3)}.pdf")

labels = ["Non-Publishable"] * 5 + ["Publishable"] * 10

#paper_names = [f"R{str(i).zfill(3)}.pdf" for i in range(1, 16)]
#labels = ["Non-Publishable"] * 5 + ["Publishable"] * 10

df = pd.DataFrame({"Paper_Name": paper_names, "Label": labels})
df.to_csv("labeled_papers.csv", index=False)

print("✅ labeled_papers.csv created!")

# Read labeled CSV
labeled_df = pd.read_csv("labeled_papers.csv")

# PDF text extraction function
def extract_text(file_path):
    try:
        reader = PdfReader(file_path)
        text = " ".join(page.extract_text() or "" for page in reader.pages)
        return text.strip()
    except Exception as e:
        print(f"❌ Failed to extract from {file_path}: {e}")
        return ""

# Embed training PDFs
print("🔧 Embedding labeled papers...")
labeled_texts = []
for _, row in labeled_df.iterrows():
    file_path = row["Paper_Name"]
    text = extract_text(file_path)
    labeled_texts.append(text)

✅ labeled_papers.csv created!
🔧 Embedding labeled papers...


In [27]:
import os
import requests
import json
import pandas as pd
from tqdm import tqdm
from sentence_transformers import util
from transformers import pipeline
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer

# Embedding model setup
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Embed labeled docs
labeled_embeddings = embedding_model.encode(labeled_texts, convert_to_tensor=True)
labels = labeled_df["Label"].tolist()

# Classify based on cosine similarity
def classify(text):
    emb = embedding_model.encode(text, convert_to_tensor=True)
    scores = util.cos_sim(emb, labeled_embeddings)[0]
    best_idx = scores.argmax().item()
    return labels[best_idx]

# Prompt template
justification_prompt = PromptTemplate.from_template(
    """You are an expert research reviewer.
Here is a paper content:
"{paper_text}"

Classify it as "Publishable" or "Non-Publishable" and give a short justification (max 100 words) for your decision.
Be specific and objective."""
)

# Google Cloud API key
GOOGLE_API_KEY = "Enter Your API KEY"

# Call PaLM API (text-bison)
def get_justification(text):
    prompt = justification_prompt.format(paper_text=text[:4000])
    url = f"https://generativelanguage.googleapis.com/v1beta2/models/text-bison-001:generateText?key={GOOGLE_API_KEY}"

    headers = {"Content-Type": "application/json"}
    data = {
        "prompt": {"text": prompt},
        "temperature": 0.7,
        "maxOutputTokens": 200,
        "topP": 0.8,
        "topK": 40
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        return response.json()["candidates"][0]["output"].strip()
    else:
        print(f"❌ API Error: {response.status_code} - {response.text}")
        return "Failed to generate justification."

# Process test papers
print("🚀 Classifying test papers...")
output = []
for paper in tqdm(os.listdir(test_pdf_dir)):
    if not paper.endswith(".pdf"):
        continue
    path = os.path.join(test_pdf_dir, paper)
    text = extract_text(path)
    if not text:
        continue
    label = classify(text)
    justification = get_justification(text)
    output.append({
        "Paper_Name": paper,
        "Classification": label,
        "Justification": justification
    })

# Save results
result_df = pd.DataFrame(output)
result_df.to_csv("test_results.csv", index=False)
print("✅ Done! Results saved to test_results.csv")



🚀 Classifying test papers...


  1%|          | 1/135 [00:01<02:25,  1.09s/it]

❌ API Error: 404 - {
  "error": {
    "code": 404,
    "message": "Requested entity was not found.",
    "status": "NOT_FOUND"
  }
}



  1%|▏         | 2/135 [00:02<02:25,  1.09s/it]

❌ API Error: 404 - {
  "error": {
    "code": 404,
    "message": "Requested entity was not found.",
    "status": "NOT_FOUND"
  }
}



...

❌ API Error: 404 - {
  "error": {
    "code": 404,
    "message": "Requested entity was not found.",
    "status": "NOT_FOUND"
  }
}



 99%|█████████▉| 134/135 [01:52<00:00,  1.10it/s]

❌ API Error: 404 - {
  "error": {
    "code": 404,
    "message": "Requested entity was not found.",
    "status": "NOT_FOUND"
  }
}



100%|██████████| 135/135 [01:54<00:00,  1.18it/s]

❌ API Error: 404 - {
  "error": {
    "code": 404,
    "message": "Requested entity was not found.",
    "status": "NOT_FOUND"
  }
}

✅ Done! Results saved to test_results.csv
